# Divergenza KL, LPPD, ELPD e LOO-CV {#sec-kl-elpd}

**Prerequisiti**

- Leggere il capitolo 7 *Ulysses’ Compass* di *Statistical Rethinking* (@McElreath_rethinking). 

**Concetti e competenze chiave**

- Comprendere in modo dettagliato la distribuzione predittiva posteriore, la divergenza $D_{\text{KL}}$, il Log-Pointwise-Predictive-Density (LPPD) e la Densità Predittiva Logaritmica Attesa (ELPD).

**Preparazione del Notebook**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
import scipy.stats as stats
from scipy.optimize import minimize
import arviz as az

In [3]:
seed: int = sum(map(ord, "kl"))
rng: np.random.Generator = np.random.default_rng(seed=seed)
az.style.use("arviz-darkgrid")
%config InlineBackend.figure_format = "retina"

## Introduzione {.unnumbered}

In questo capitolo, esamineremo in dettaglio quattro concetti fondamentali per la valutazione e il confronto di modelli statistici nel contesto bayesiano: la distribuzione predittiva posteriore, la divergenza $D_{\text{KL}}$, il Log-Pointwise-Predictive-Density (LPPD) e la Densità Predittiva Logaritmica Attesa (Expected Log Predictive Density, ELPD). Questi strumenti ci permettono di quantificare l'adattamento dei modelli ai dati e la loro capacità predittiva.

## Distribuzione Predittiva Posteriore

La distribuzione predittiva posteriore $q(\tilde{y} \mid y)$ rappresenta la distribuzione dei possibili nuovi dati $\tilde{y}$, alla luce dei dati osservati $y$. Essa si ottiene combinando:

- la distribuzione del modello per i nuovi dati data una configurazione dei parametri $\theta$: $q(\tilde{y} \mid \theta)$;
- la distribuzione posteriore dei parametri dati i dati osservati: $p(\theta \mid y)$.

Matematicamente, la distribuzione predittiva posteriore può essere scritta come:

$$
q(\tilde{y} \mid y) = \int q(\tilde{y} \mid \theta) p(\theta \mid y) d\theta.
$$

Questa espressione si legge come l'integrale della distribuzione predittiva $q(\tilde{y} \mid \theta)$ pesata dalla distribuzione posteriore dei parametri $p(\theta \mid y)$. In altre parole, stiamo "mediando" le previsioni del modello su tutte le possibili configurazioni dei parametri, tenendo conto della loro probabilità posteriore. La distribuzione predittiva posteriore è stata descritta nella @sec-posterior-predictive-distribution.

## Divergenza di Kullback-Leibler $D_{\text{KL}}$

La divergenza di Kullback-Leibler, $D_{\text{KL}}$, misura la "distanza" tra due distribuzioni di probabilità, $P$ e $Q$. Essa è definita come:

$$
D_{\text{KL}}(P \parallel Q) = \mathbb{E}_P[\log p(X)] - \mathbb{E}_P[\log q(X)],
$$

dove $\mathbb{E}_P[\log p(X)]$ rappresenta il valore atteso del logaritmo della distribuzione $p(X)$ rispetto a $P$ stessa, e $\mathbb{E}_P[\log q(X)]$ è il valore atteso del logaritmo della distribuzione $q(X)$ rispetto alla distribuzione $P$.

Per variabili discrete, il valore atteso $\mathbb{E}_P[\log p(X)]$ è calcolato come:

$$
\mathbb{E}_P[\log p(X)] = \sum_{i=1}^n p_i \log p_i,
$$

dove $p_i = p(x_i)$ è la probabilità associata al valore $x_i$.

Nel caso di variabili continue, questa espressione diventa:

$$
\mathbb{E}_P[\log p(X)] = \int p(x) \log p(x) \, dx,
$$

che rappresenta l'entropia negativa della distribuzione $p$.

Il termine $\mathbb{E}_P[\log q(X)]$ rappresenta il valore atteso del logaritmo della distribuzione $q(X)$ sotto la distribuzione $p(X)$:

$$
\mathbb{E}_P[\log q(X)] = \sum_{i=1}^n p_i \log q_i
$$

per variabili discrete, e:

$$
\mathbb{E}_P[\log q(X)] = \int p(x) \log q(x) \, dx
$$

per variabili continue. Questo termine rappresenta l'entropia incrociata tra $P$ e $Q$.

Quindi, la divergenza $D_{\text{KL}}$ può essere espressa per variabili discrete come:

$$
D_{\text{KL}}(P \parallel Q) = \sum_{i=1}^n p_i (\log p_i - \log q_i),
$$

e per variabili continue come:

$$
D_{\text{KL}}(P \parallel Q) = \int p(x) (\log p(x) - \log q(x)) \, dx.
$$

Un punto importante è che la divergenza $D_{\text{KL}}$ non è simmetrica, cioè $D_{\text{KL}}(P \parallel Q) \neq D_{\text{KL}}(Q \parallel P)$. Questa asimmetria riflette il fatto che la divergenza misura quanta "informazione" si perde usando la distribuzione $Q$ al posto della distribuzione vera $P$.

::: {#exm-}

Per illustrare meglio il concetto di divergenza di Kullback-Leibler $D_{\text{KL}}$ nel caso discreto, consideriamo un esempio semplice con due distribuzioni di probabilità discrete, $P$ e $Q$, definite su un insieme di eventi $\{A, B, C\}$. Supponiamo che le probabilità associate a ciascun evento sotto le distribuzioni $P$ e $Q$ siano le seguenti:

- Distribuzione $P$: $p(A) = 0.5$, $p(B) = 0.3$, $p(C) = 0.2$
- Distribuzione $Q$: $q(A) = 0.4$, $q(B) = 0.4$, $q(C) = 0.2$

La divergenza di Kullback-Leibler $D_{\text{KL}}(P \parallel Q)$ è definita come:

$$
D_{\text{KL}}(P \parallel Q) = \sum_{i} p(i) \log \frac{p(i)}{q(i)},
$$

dove $i$ scorre su tutti gli eventi possibili $\{A, B, C\}$. In questo caso, dobbiamo calcolare i seguenti termini: $p(A) \log \frac{p(A)}{q(A)}$, $p(B) \log \frac{p(B)}{q(B)}$, e $p(C) \log \frac{p(C)}{q(C)}$.

Calcoliamo ciascun termine:

$$
p(A) \log \frac{p(A)}{q(A)} = 0.5 \log \frac{0.5}{0.4} \approx 0.11155,
$$

$$
p(B) \log \frac{p(B)}{q(B)} = 0.3 \log \frac{0.3}{0.4} \approx -0.08631,
$$

$$
p(C) \log \frac{p(C)}{q(C)} = 0.2 \log \frac{0.2}{0.2} = 0.
$$

Ora, sommiamo tutti i termini per ottenere la divergenza di Kullback-Leibler:

$$
D_{\text{KL}}(P \parallel Q) = 0.11155 + (-0.08631) + 0 = 0.02524.
$$

Quindi, la divergenza KL $D_{\text{KL}}(P \parallel Q) \approx 0.025$ indica che la distribuzione $Q$ è relativamente vicina alla distribuzione $P$, con una piccola discrepanza tra le due. Un valore più elevato di $D_{\text{KL}}$ suggerirebbe una maggiore differenza tra le distribuzioni $P$ e $Q$, indicando una maggiore perdita di informazione se si utilizza $Q$ al posto di $P$.

:::

## Problema con $p$ Sconosciuto

Il problema principale nella pratica statistica è che la distribuzione vera dei dati, indicata con $p(x)$, non è nota. Se conoscessimo la distribuzione $p(x)$, non sarebbe necessario fare inferenza statistica. Tuttavia, il nostro obiettivo è confrontare diversi modelli, come $q(x)$ e $r(x)$, per determinare quale di essi si avvicina di più a $p(x)$.

### L'Idea della Divergenza Relativa

Fortunatamente, per confrontare due modelli $q(x)$ e $r(x)$, non è necessario conoscere esattamente la distribuzione $p(x)$. Possiamo confrontare i modelli in termini di divergenza relativa da $p(x)$. In pratica, quando confrontiamo le divergenze $D_{\text{KL}}(p \parallel q)$ e $D_{\text{KL}}(p \parallel r)$, molte componenti legate a $p(x)$ si annullano, perché stiamo considerando la differenza tra le due divergenze. Questa differenza dipende principalmente dalla differenza tra $q(x)$ e $r(x)$, non dalla distribuzione $p(x)$ stessa.

### Log-Probability Score (Log-Score)

Il **log-probability score** (o **log-score**) è una misura pratica che possiamo utilizzare per valutare i modelli in assenza della conoscenza della distribuzione vera $p(x)$. Per ogni modello $q(x)$, il log-score è calcolato come:

$$
S(q) = \sum_{i} \log(q(x_i)),
$$

dove $q(x_i)$ rappresenta la probabilità predetta dal modello $q(x)$ per l'evento $x_i$.

Questo score rappresenta una stima dell'aspettativa del logaritmo delle probabilità predette dal modello $q(x)$ rispetto ai dati osservati. In altre parole, misura quanto bene il modello $q(x)$ predice i dati osservati. Un log-score più alto indica un modello con una migliore accuratezza predittiva, poiché il modello assegnerebbe una probabilità maggiore agli eventi che si verificano effettivamente.

::: {#exm-}

Consideriamo un semplice esempio numerico per illustrare il calcolo del *log-probability score* (o *log-score*) per un modello $q$. Immaginiamo di avere un piccolo dataset con 3 osservazioni, e che il modello $q$ predica le probabilità per ciascuna osservazione come segue:

- Osservazione 1: $q_1 = 0.8$,
- Osservazione 2: $q_2 = 0.6$,
- Osservazione 3: $q_3 = 0.7$.

Il *log-score* si calcola sommando i logaritmi delle probabilità predette:

$$
S(q) = \log(0.8) + \log(0.6) + \log(0.7) \approx -0.2231 + (-0.5108) + (-0.3567) = -1.0906.
$$

Il log-score totale per questo modello $q$ è $-1.0906$. Poiché un log-score più alto (meno negativo) indica una migliore accuratezza predittiva, questo risultato suggerisce che $q$ ha una discreta accuratezza per le osservazioni date. Un modello $r$ con un log-score più alto sarebbe preferibile in termini di accuratezza predittiva rispetto a $q$.

:::

::: {#exm-}

Per chiarire ulteriormente il concetto di *log-probability score*, possiamo applicarlo al caso di un modello di regressione. In questo contesto, il calcolo del *log-probability score* implica la valutazione della probabilità che il modello assegna a ciascuna osservazione, considerando la distribuzione degli errori associata al modello.

Consideriamo un modello di regressione lineare semplice:

$$
y_i = \beta_0 + \beta_1 x_i + \epsilon_i,
$$

dove $y_i$ è la variabile dipendente, $x_i$ è la variabile indipendente, $\beta_0$ e $\beta_1$ sono i coefficienti del modello, e $\epsilon_i$ è l'errore, tipicamente assunto come distribuito secondo una normale $\mathcal{N}(0, \sigma^2)$.

Per calcolare la probabilità di ciascuna osservazione $y_i$ data la predizione del modello, usiamo la distribuzione degli errori. Se $y_i$ è distribuito secondo una normale con media $\hat{y}_i = \beta_0 + \beta_1 x_i$ e varianza $\sigma^2$, allora la probabilità di osservare $y_i$ è:

$$
p(y_i \mid x_i) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(y_i - \hat{y}_i)^2}{2\sigma^2}\right).
$$

Il *log-score* per ciascuna osservazione è il logaritmo della probabilità predetta:

$$
\log p(y_i \mid x_i) = -\frac{1}{2} \log(2\pi\sigma^2) - \frac{(y_i - \hat{y}_i)^2}{2\sigma^2}.
$$

Il log-score totale del modello è la somma dei log-score su tutte le osservazioni:

$$
S(q) = \sum_{i=1}^n \log p(y_i \mid x_i) = -\frac{n}{2} \log(2\pi\sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^n (y_i - \hat{y}_i)^2.
$$

- *Primo termine*: $-\frac{n}{2} \log(2\pi\sigma^2)$ è una costante che dipende dal numero di osservazioni $n$ e dalla varianza degli errori $\sigma^2$.
- *Secondo termine*: $-\frac{1}{2\sigma^2} \sum_{i=1}^n (y_i - \hat{y}_i)^2$ è proporzionale alla somma dei quadrati degli errori (SSE), che misura quanto le predizioni si discostano dai valori osservati.

Il log-score fornisce quindi una misura dell'accuratezza del modello di regressione, con valori più alti (meno negativi) che indicano un modello migliore. In pratica, poiché il primo termine è una costante, la differenza nei log-score tra modelli diversi sarà determinata principalmente dal secondo termine, legato alla SSE.

In sintesi, nel contesto di un modello di regressione, la probabilità di ciascuna osservazione si basa sulla distribuzione normale degli errori, e il log-score riflette quanto bene il modello predice i dati osservati, penalizzando modelli con errori più grandi.

:::

### Log-Pointwise-Predictive-Density (LPPD)

Il *Log-Pointwise-Predictive-Density* (LPPD) è una versione bayesiana del log-score, che tiene conto dell'incertezza sui parametri del modello. In un contesto bayesiano, non abbiamo un solo set di parametri $\theta_s$, ma una distribuzione posteriore su di essi. Pertanto, il log-score viene calcolato come una media logaritmica su questa distribuzione. Il LPPD è una misura che deriva direttamente dalla distribuzione predittiva posteriore e rappresenta la somma delle densità predittive logaritmiche calcolate per ogni osservazione, mediate sulla distribuzione posteriore dei parametri. Formalmente, si definisce come:

$$
\text{LPPD} = \sum_{i=1}^n \log \left(\frac{1}{S} \sum_{s=1}^{S} p(y_i \mid \theta_s)\right),
$$

dove:

- $y_i$ è l'i-esima osservazione,
- $S$ è il numero di campioni dalla distribuzione posteriore,
- $\theta_s$ è un campione di parametri dalla distribuzione posteriore.

Il LPPD tiene conto dell'incertezza nei parametri, calcolando il logaritmo della densità predittiva media per ogni punto dati, mediata su tutte le possibili configurazioni dei parametri posteriore. Questo approccio è fondamentale per catturare la reale capacità predittiva di un modello, tenendo conto della variabilità nei parametri.

In conclusione, il log-score e il LPPD sono metodi per stimare la bontà di un modello rispetto ai dati osservati, senza richiedere la conoscenza esatta di $p$. Anche se non possiamo calcolare direttamente la divergenza $D_{\text{KL}}$ perché $p$ è sconosciuto, possiamo comunque confrontare diversi modelli basandoci su queste misure. La differenza tra i log-score (o LPPD) di due modelli $q$ e $r$ ci dà un'informazione su quale modello è "più vicino" alla verità, analogamente a come useremmo la divergenza $D_{\text{KL}}$ se conoscessimo $p$.

## Expected Log Predictive Density (ELPD)

L'*Expected Log Predictive Density* (ELPD) è una misura che valuta la capacità predittiva di un modello utilizzando la tecnica della cross-validation. Si definisce come:

$$
\text{ELPD} = \sum_{i=1}^n \log p(y_i \mid \mathbf{y}_{-i}),
$$

dove:

- $y_i$ è l'i-esima osservazione,
- $\mathbf{y}_{-i}$ rappresenta tutte le osservazioni eccetto $y_i$.

L'ELPD utilizza la *Leave-One-Out Cross-Validation* (LOO-CV) per stimare la densità predittiva logaritmica di ogni osservazione, escludendo quella stessa osservazione dal training. Questo metodo permette di evitare l'overfitting, valutando la performance del modello su dati non visti.

::: {#exm-}

Per illustrare il calcolo dell'ELPD, vediamo un esempio semplice con un set di dati molto piccolo. Supponiamo di avere un dataset di tre osservazioni: $y_1, y_2, y_3$. Supponiamo che il nostro modello stimi le probabilità per ciascuna osservazione in base a tutte le altre osservazioni, cioè utilizziamo la leave-one-out cross-validation (LOO-CV) per calcolare $p(y_i \mid \mathbf{y}_{-i})$.

Immaginiamo che il modello produca le seguenti probabilità condizionali per ogni osservazione $y_i$:

- $p(y_1 \mid y_2, y_3) = 0.6$,
- $p(y_2 \mid y_1, y_3) = 0.7$,
- $p(y_3 \mid y_1, y_2) = 0.5$.

L'ELPD si calcola sommando i logaritmi di queste probabilità:

$$
\text{ELPD} = \log p(y_1 \mid y_2, y_3) + \log p(y_2 \mid y_1, y_3) + \log p(y_3 \mid y_1, y_2).
$$

Calcoliamo i logaritmi naturali di ciascuna probabilità:

- $\log p(y_1 \mid y_2, y_3) = \log 0.6 \approx -0.5108$,
- $\log p(y_2 \mid y_1, y_3) = \log 0.7 \approx -0.3567$,
- $\log p(y_3 \mid y_1, y_2) = \log 0.5 \approx -0.6931$.

Sommiamo i logaritmi per ottenere l'ELPD:

$$
\text{ELPD} = -0.5108 + (-0.3567) + (-0.6931) = -1.5606.
$$

L'ELPD ottenuto è $-1.5606$. In generale, valori più vicini a 0 o positivi indicano una migliore capacità predittiva del modello, poiché suggeriscono che le probabilità condizionali assegnate dal modello alle osservazioni lasciate fuori non sono troppo basse. Valori molto negativi indicherebbero che il modello ha assegnato probabilità molto basse alle osservazioni effettivamente osservate, suggerendo una scarsa capacità predittiva. L'ELPD è un modo efficace per valutare quanto bene un modello generalizza a nuovi dati, evitando l'overfitting.

:::

## Collegamento tra Divergenza $D_{\text{KL}}$ e ELPD

Entrambe le misure (divergenza $D_{\text{KL}}$ e ELPD) valutano la qualità del modello, ma da prospettive diverse:

- la divergenza $D_{\text{KL}}$ misura quanto la distribuzione predittiva del modello $Q$ si avvicina alla "vera" distribuzione $P$;
- l'ELPD valuta direttamente la capacità del modello di prevedere nuovi dati, utilizzando la tecnica leave-one-out.

L'ELPD tende a favorire modelli che non solo si adattano bene ai dati osservati, ma che sono anche robusti rispetto all'overfitting, grazie alla sua enfasi sulla capacità predittiva su dati non osservati.

In sintesi, la distribuzione predittiva posteriori, la divergenza $D_{\text{KL}}$ e l'ELPD sono strumenti matematici che ci permettono di valutare la bontà di un modello statistico. La divergenza KL fornisce una misura teoricamente ideale di quanto un modello approssima la vera distribuzione dei dati, mentre l'ELPD offre un'alternativa praticabile che valuta la capacità del modello di fare previsioni su nuovi dati. Questi concetti sono fondamentali nell'inferenza bayesiana, dove è importante non solo adattarsi ai dati esistenti, ma anche generalizzare bene su dati futuri.

::: {#exm-}

Consideriamo un secondo esempio per illustrare il concetto di ELPD utilizzando la distribuzione binomiale. Immaginiamo di condurre un esperimento in cui lanciamo una moneta 10 volte e contiamo il numero di volte in cui otteniamo testa. Supponiamo che la vera probabilità di ottenere testa sia 0.6.

1) *Distribuzione reale dei dati:* Segue una distribuzione binomiale con 10 lanci e probabilità di successo pari a 0.6: $y \sim \text{Binomiale}(10, 0.6).$
2) *Distribuzione stimata dal modello:* Il nostro modello ipotizza che la probabilità di ottenere testa sia 0.5, cioè considera la moneta come equa: $p(y \mid \theta) = \text{Binomiale}(10, 0.5).$

Ora procediamo al calcolo dell'ELPD.

In [4]:
# Parametri
n = 10  # numero di lanci
p = 0.6  # vera probabilità di testa
q = 0.5  # probabilità stimata dal modello

# Calcolo ELPD
elpd = 0
for y in range(n + 1):
    # Probabilità di y secondo la vera distribuzione
    p_y = binom.pmf(y, n, p)

    # Log della densità predittiva del modello
    log_q_y = binom.logpmf(y, n, q)

    # Somma pesata
    elpd += p_y * log_q_y

print(f"ELPD del modello che stima p=0.5: {elpd:.4f}")

# Per confronto, calcoliamo l'ELPD per il modello "vero"
elpd_true = 0
for y in range(n + 1):
    p_y = binom.pmf(y, n, p)
    log_p_y = binom.logpmf(y, n, p)
    elpd_true += p_y * log_p_y

print(f"ELPD del modello vero (con p=0.6): {elpd_true:.4f}")

ELPD del modello che stima p=0.5: -2.0549
ELPD del modello vero (con p=0.6): -1.8536


La conclusione è che l'ELPD del modello vero è maggiore (meno negativo) rispetto a quello del nostro modello stimato, il che riflette una capacità predittiva superiore del modello vero.

Questo esempio dimostra come l'ELPD quantifica la capacità predittiva di un modello:

1. Si considerano tutti i possibili risultati dell'esperimento (da 0 a 10 teste).
2. Per ciascun risultato, si calcola:

   - La probabilità di osservare quel risultato secondo la distribuzione reale.
   - Il logaritmo della densità predittiva del modello stimato $q$ per lo stesso risultato.
3. Si moltiplicano questi due valori e si sommano i risultati per tutti i possibili esiti.

In sintesi, l'ELPD permette di confrontare l'efficacia predittiva di diversi modelli: un valore più alto (meno negativo) indica una migliore capacità del modello di prevedere i dati osservati. Nell'esempio presentato, il modello vero (con $p = 0.6$) ha un ELPD superiore rispetto al modello stimato (con $p = 0.5$), confermando che il primo ha una capacità predittiva migliore.

:::

### Collegamento tra LPPD e ELPD

Il LPPD e l'ELPD sono strettamente collegati e forniscono una valutazione robusta della capacità predittiva di un modello statistico. Il LPPD fornisce una stima del log-score basata sull'intera distribuzione posteriore dei parametri, mentre l'ELPD utilizza la cross-validation per stimare la capacità predittiva del modello su nuovi dati.

Sebbene il LPPD sia una buona misura, esso tende a migliorare con l'aumentare della complessità del modello, a causa del fenomeno dell'overfitting. Il modello, infatti, potrebbe adattarsi perfettamente ai dati di addestramento, ma non generalizzare bene ai nuovi dati, catturando il "rumore" presente nei dati piuttosto che il vero segnale sottostante.

### Leave-One-Out Cross-Validation (LOO-CV)

La *cross-validation*, in particolare la *Leave-One-Out Cross-Validation (LOO-CV)*, è una tecnica per affrontare questo problema. La LOO-CV valuta un modello lasciando fuori una sola osservazione dal dataset, addestrando il modello sul resto dei dati e poi testandolo sull'osservazione esclusa. Questo processo viene ripetuto per ogni osservazione nel dataset, e il risultato finale è una media delle prestazioni del modello su tutte le osservazioni lasciate fuori.

Come la divergenza $D_{\text{KL}}$, anche il calcolo dell'ELPD richiede, in teoria, la conoscenza della vera distribuzione $p$, che è ignota. Tuttavia, a differenza della divergenza $D_{\text{KL}}$, disponiamo di metodi pratici per approssimare l'ELPD senza la necessità di conoscere la vera distribuzione dei dati. Uno dei metodi più robusti e comunemente utilizzati per questa stima è la Leave-One-Out Cross-Validation (LOO-CV). Questo metodo consiste nel rimuovere una singola osservazione dal dataset, adattare il modello sui dati rimanenti, e poi valutare la densità predittiva per l'osservazione esclusa. Questa procedura viene ripetuta per ogni osservazione, e i risultati vengono sommati per ottenere una stima complessiva dell'ELPD.

La LOO-CV è considerata uno dei metodi migliori per la selezione del modello perché fornisce una stima dell'accuratezza predittiva del modello su dati che non sono stati utilizzati per addestrarlo. Questo approccio riduce il rischio di overfitting, poiché misura come il modello si comporta su dati effettivamente "nuovi".

Quindi, sebbene il log-probability score sia una buona misura, il suo miglioramento con l'aumentare della complessità del modello può essere mitigato utilizzando tecniche di cross-validation, come la LOO-CV. Questo metodo fornisce una stima accurata della performance del modello su dati non visti, evitando così il problema dell'overfitting.

### Criteri di Informazione come Approssimazioni alla Divergenza $D_{\text{KL}}$

Oltre al LOO-CV, sono stati proposti altri metodi per approssimare l'ELPD, che derivano direttamente dal concetto di divergenza $D_{\text{KL}}$. Tra questi, i più noti sono i criteri di informazione, come l'errore quadratico medio (MSE), l'Akaike Information Criterion (AIC), il Bayesian Information Criterion (BIC) e il Widely Applicable Information Criterion (WAIC). Questi criteri forniscono approcci alternativi per valutare i modelli, bilanciando la bontà di adattamento del modello ai dati osservati con la sua complessità.

#### Errore Quadratico Medio (MSE)

L'Errore Quadratico Medio (Mean Squared Error o MSE) misura la discrepanza media tra le previsioni del modello e i valori reali:

$$ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2, $$

dove:

- $n$ è il numero totale di osservazioni,
- $y_i$ sono i valori reali,
- $\hat{y}_i$ sono i valori previsti dal modello.

Un MSE inferiore indica un migliore adattamento del modello ai dati.

#### Criterio di Informazione di Akaike (AIC)

Il Criterio di Informazione di Akaike (AIC) va oltre l'MSE, considerando sia l'adattamento del modello che la sua complessità:

$$ AIC = -2 \sum \log p(y_i \mid \hat{\theta}_{\text{mle}}) + 2k, $$

dove:

- $\hat{\theta}_{\text{mle}}$ sono i parametri stimati del modello,
- $k$ è il numero di parametri del modello.

L'AIC bilancia la bontà di adattamento (primo termine) con la complessità del modello (secondo termine). Un valore più basso di AIC indica una minor perdita di informazione, suggerendo un modello preferibile.

Vantaggi e limitazioni:

- facile e veloce da calcolare;
- può essere meno accurato per campioni piccoli o modelli complessi;
- fornisce un'approssimazione asintoticamente corretta dell'ELPD per modelli regolari e campioni grandi.

#### Criterio di Informazione Bayesiano (BIC)

Il Criterio di Informazione Bayesiano (BIC) è definito come:

$$
BIC = \ln(n)k - 2\ln(L),
$$

dove:

- $n$ è il numero di osservazioni,
- $k$ è il numero di parametri del modello,
- $L$ è il valore massimo della funzione di verosimiglianza del modello.

Il termine $\ln(L)$ rappresenta il logaritmo naturale della verosimiglianza massima, che indica quanto bene il modello si adatta ai dati osservati.
  
Il BIC impone una penalità maggiore per l'incremento dei parametri, rendendolo particolarmente adeguato per dataset di grandi dimensioni.

#### Widely Applicable Information Criterion (WAIC)

Il WAIC è una versione avanzata dell'AIC, particolarmente utile nel contesto bayesiano. Considera l'intera distribuzione a posteriori dei parametri anziché solo la stima puntuale:

$$ 
WAIC = -2\left[ \sum_{i=1}^{n} \log \left( \frac{1}{S} \sum_{s=1}^{S} p(y_i \mid \theta^{(s)}) \right) - \sum_{i=1}^{n} \text{Var}_{\theta^{(s)}} \left( \log p(y_i \mid \theta^{(s)}) \right) \right], 
$$

dove:

- $S$ è il numero di campioni dalla distribuzione a posteriori,
- $\text{Var}_{\theta^{(s)}}$ è la varianza della log-verosimiglianza.

Caratteristiche del WAIC:

1. calcola il logaritmo della densità predittiva per ogni punto dati;
2. penalizza la complessità del modello basandosi sulla variabilità delle sue predizioni;
3. la somma delle varianze a posteriori del logaritmo della densità predittiva converge al numero effettivo di parametri del modello.

## Riflessioni Conclusive

La valutazione dei modelli statistici è cruciale per garantire la loro affidabilità e capacità di generalizzazione. Quando si sviluppano modelli, è fondamentale utilizzare strumenti che permettano di confrontare diverse alternative e selezionare quella più adatta ai dati disponibili, assicurando previsioni accurate.

Uno strumento chiave in questo processo è la *Expected Log Predictive Density* (ELPD), che valuta l'accuratezza con cui un modello può prevedere nuovi dati. Tuttavia, il calcolo diretto dell'ELPD è spesso impraticabile, poiché richiederebbe la conoscenza esatta del meccanismo generatore dei dati, $p_t(y)$. Per ovviare a questa limitazione, si utilizza una stima approssimativa basata sulla distribuzione predittiva a posteriori del modello, $p(\tilde{y} \mid y)$.

Per ottenere una valutazione accurata della capacità di generalizzazione di un modello su futuri set di dati, si ricorre a metodi di validazione incrociata, come la *Leave-One-Out Cross-Validation* (LOO-CV). Questa tecnica implica l'addestramento del modello su un sottoinsieme dei dati e il suo test su un altro, isolando così le prestazioni del modello dalle variazioni casuali presenti nei dati. L'indice LOO-CV risultante è fondamentale per confrontare diversi modelli.

La differenza nei valori di LOO-CV tra due modelli, insieme al calcolo dell'errore standard associato a questa differenza, ci permette di determinare se esistono differenze significative nelle prestazioni dei modelli. Se il rapporto tra la differenza di LOO-CV e il relativo errore standard è superiore a 2, possiamo concludere che le differenze osservate non sono casuali, ma riflettono una vera superiorità di un modello rispetto all'altro.

Nel contesto della statistica bayesiana, il *Log Pointwise Predictive Density* (LPPD) e l'*Expected Log Pointwise Predictive Density* (ELPD) sono strumenti essenziali per la valutazione dei modelli. Sebbene la divergenza $D_{\text{KL}}$ rappresenti una misura ideale per quantificare la distanza tra la distribuzione di probabilità reale dei dati e quella stimata dal modello, la sua applicazione è limitata dal fatto che la distribuzione vera dei dati è generalmente sconosciuta.

A differenza della divergenza $D_{\text{KL}}$, il LPPD offre un approccio pratico per valutare la capacità predittiva dei modelli, anche se può soffrire di overfitting. Per mitigare questo problema, si utilizza l'ELPD, che stima la capacità del modello di generalizzare oltre i dati di addestramento.

L'ELPD e la divergenza $D_{\text{KL}}$ sono strumenti complementari: l'ELPD misura la capacità predittiva su nuovi dati, con valori più alti che indicano una maggiore capacità di generalizzazione, mentre la divergenza $D_{\text{KL}}$ quantifica la differenza tra la distribuzione vera dei dati e quella stimata dal modello, con valori più bassi che indicano una migliore approssimazione della distribuzione reale.

Esiste una relazione diretta tra ELPD e divergenza $D_{\text{KL}}$: massimizzare l'ELPD equivale a minimizzare la divergenza $D_{\text{KL}}$, poiché entrambi gli approcci mirano a sviluppare modelli che rappresentino accuratamente la realtà dei dati.

In sintesi, la divergenza $D_{\text{KL}}$ valuta l'adattamento del modello ai dati osservati, mentre l'ELPD e i suoi metodi di approssimazione, come il LOO-CV, misurano la capacità del modello di generalizzare su dati futuri, offrendo una valutazione più completa della qualità del modello. La selezione del modello ottimale richiede un equilibrio tra adattamento ai dati e semplicità. Utilizzando tecniche di validazione incrociata e criteri di informazione come AIC, BIC e WAIC, è possibile costruire modelli che si adattano bene ai dati osservati, forniscono previsioni affidabili su nuovi dati e catturano le tendenze rilevanti senza essere eccessivamente influenzati dal rumore.

## Informazioni sull'Ambiente di Sviluppo {.unnumbered}

In [25]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Fri Jul 26 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

arviz     : 0.18.0
matplotlib: 3.9.1
numpy     : 1.26.4
scipy     : 1.14.0

Watermark: 2.4.3

